# Notebook pour le calcul des alertes (synthèses communales)

### Initialisation contexte test appli

In [ ]:
import os

# Manually filling Config dictionnary (can be filled by reading Config_process.env) :

CONFIG = {}

CONFIG['WRK_DIR'] = os.path.normpath('D:/MATHIS/0_Projet_Secheresse/1_Scripts/toolbox/eo4dm-oeil/EO4DM')
os.chdir(CONFIG['WRK_DIR'])
CONFIG['WRK_DIR'] = os.path.normpath('Y:/EO4DM')
CONFIG['DATA_HISTO'] = os.path.join(CONFIG['WRK_DIR'],'DATA_HISTO')
CONFIG['ANNEX_DIR'] = os.path.join(CONFIG['WRK_DIR'],'ANNEX')

CONFIG['TERRITORY'] = 'New Caledonia (Fr)'
CONFIG['PERIOD_START'] = '2024-01-01'
CONFIG['PERIOD_END'] = '2024-02-01'
CONFIG['DRIVE_FOLDER'] = 'EO4DM_EXPORT_NOTEBOOK'
CONFIG['CLEAN_RUNFOLDER'] = 0                         # if 1, drought already existing folder is cleaned (deleted, and re-created)
CONFIG['MODE'] = 'DROUGHT'                                # 'AUTO', 'MANUAL', 'INDICES' or 'DROUGHT' mode of drought processing chain
CONFIG['DROUGHT_STATS'] = 1                               # if 1, drought spatial stats are estimated on all territory and sub-area
CONFIG['KEY_STATS'] = ''

CONFIG['CRS_OUT_modis'] = 'EPSG:4326'
CONFIG['lon_min_modis'] = 163.46601498126984
CONFIG['lon_max_modis'] = 168.29999935626984
CONFIG['lat_min_modis'] = -22.85859364271164
CONFIG['lat_max_modis'] =-19.44843739271164


### Création arborescence dossiers DATA_HISTO et controle ANNEX_DIR

In [ ]:
import dmpipeline.ALERT_Processing.ALERT_main_functions as alertmain
alertmain.prepare_DATAFolders(CONFIG)

### Test to collect last ASCAT SWI from Copernicus server

In [ ]:
import dmpipeline.ALERT_Processing.ALERT_data_access as alertaccess
alertaccess.download_newProducts(CONFIG)


### Test détection auto de la période à traiter (en fonction de data_histo)

In [ ]:
import dmpipeline.ALERT_Processing.ALERT_main_functions as alertmain
go_process, CONFIG['PERIOD_START'], CONFIG['PERIOD_END'] = alertmain.auto_processingPERIOD(CONFIG)

print(f'go_process = {go_process}')
PERIOD_START = CONFIG['PERIOD_START']
PERIOD_END = CONFIG['PERIOD_END']
print(f'PERIOD_START = {PERIOD_START}')
print(f'PERIOD_END = {PERIOD_END}')

### Test to CHECK/PREPARE all products

In [ ]:
# Control ASCAT products availability updates PERIOD START/END :
import dmpipeline.ALERT_Processing.ALERT_main_functions as alertmain
go_ascat, COLLECTION_ASCAT, PERIOD_START_NEW, PERIOD_END_NEW = alertmain.Check_Products_ASCAT(CONFIG)
print(f'go_ascat = {go_ascat}')
print(f'NEW PERIOD_START = {PERIOD_START_NEW}')
print(f'NEW PERIOD_END = {PERIOD_END_NEW}')
CONFIG['PERIOD_START'] = PERIOD_START_NEW
CONFIG['PERIOD_END'] = PERIOD_END_NEW

# Control METEO products availability :
go_meteo = alertmain.Check_Products_METEO(CONFIG)
print(f'go_meteo = {go_meteo}')

# # Control VHI products availability :
go_vhi = alertmain.Check_Products_VHI(CONFIG)
print(f'go_vhi = {go_vhi}')


### Full Test For Checking products

In [ ]:
(go_products, COLLECTION_ASCAT, CONFIG['PERIOD_START'], CONFIG['PERIOD_END'],
 CONFIG['TERRITORY'], OUTDIR_PATHS) = alertmain.CheckInit_Products_ALERT(CONFIG)
PERIOD_START_NEW = CONFIG['PERIOD_START']
PERIOD_END_NEW = CONFIG['PERIOD_END']
TERRITORY = CONFIG['TERRITORY']
print(f'go_products = {go_products}')
print(f'NEW PERIOD_START = {PERIOD_START_NEW}')
print(f'NEW PERIOD_END = {PERIOD_END_NEW}')
print(f'TERRITORY = {TERRITORY}')

### Test to initialize ASCAT processing (extracting paths to local products)

In [ ]:
alertmain.processingASCAT(CONFIG, OUTDIR_PATHS, COLLECTION_ASCAT)

### Test to process Moisture Deficit MAI

In [ ]:
alertmain.process_MoistureDeficit_MAI(CONFIG, OUTDIR_PATHS)

### Création dossier de sortie RUN

In [ ]:
import dmpipeline.ALERT_Processing.ALERT_main_functions as alertmain

OUTDIR_PATHS = alertmain.prepare_RUNFolders(CONFIG)

### Test to process DROUGHT ALERTS

In [ ]:
alertmain.process_DroughtAlert(CONFIG, OUTDIR_PATHS)